# Selenium

In [1]:
# @title Importing Libraries

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from config import *
import time
driver = webdriver.Chrome()


## Driver startup
The following code will go to twitter and accept the cookies

In [2]:
#get twitter.com
driver.get("https://twitter.com/") #driver.get returns after the page is loaded but just to be sure, I'll wait 5 more seconds
print("Waiting for 5 seconds")
time.sleep(5)
print("Done Waiting")
#click on the full xpath to accept cookies
driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div/div[2]/div[1]').click()


Waiting for 5 seconds
Done Waiting


In [3]:
#capture all the elements whose tag is 'article'
def getTweets(tagName = 'article'):
    tweets = driver.find_elements(By.TAG_NAME, tagName)
    return tweets

In [4]:
def getTweetContent(tweet):
    return " ".join(tweet.text.split('\n')[4:-4])

In [5]:
def printCurrentTweets(tweets = []):
    """
    This function prints the current tweets on the page that can be located through the tag name 'article'
    """
    tweets = tweets if len(tweets) > 0 else getTweets()
    for tweet in tweets:
        body = getTweetContent(tweet)
        print(body)
        print("--------------------------------------------------")
        
        

    

In [6]:
def colorBackground(tweet, color = "red"):
    
    driver.execute_script("arguments[0].style.backgroundColor = '"+color+"';", tweet)

In [7]:
tweets = getTweets()

In [8]:
printCurrentTweets()

Truly an iconic moment for the girls. #FRIENDS 1:04
--------------------------------------------------
German Shepherd babysitting the Dobermans
--------------------------------------------------
Cat bus
--------------------------------------------------
Incredible moment captured on video as a shark takes an injured sea turtle to humans on a boat so they can help save it  From  Alvin Foo
--------------------------------------------------
we went to a new park today. and right at the entrance. was a big bucket of tennis balls with a little sign. that the human said was there in memory of a very good dog. who really liked tennis balls. and that i could play with one. to help that very good dog be remembered
--------------------------------------------------
did he just lay an egg
--------------------------------------------------
A small reminder that people only fear sharks when dolphins are worse. You are more likely to get sexually assaulted by a dolphin than a shark. They torture ot

# Neural Network

In [9]:
import tensorflow  as  tf
from  tensorflow  import  keras

from tensorflow.keras import layers
from keras.utils import pad_sequences


from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import sklearn

import numpy as  np
import matplotlib.pyplot  as  plt
import os

import random

import pickle5 as pickle

import pandas  as  pd
import tqdm as tqdm

import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


#import  the confusion matrix
from sklearn.metrics import confusion_matrix


from keras.preprocessing import sequence


2023-05-31 00:45:35.238132: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-31 00:45:35.258653: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-31 00:45:35.361414: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-31 00:45:35.362042: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 00:45:36.097510: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

## Load Models

In [10]:
#load the ./disasterModel.h5 model using keras
model = keras.models.load_model('./models/disasterModel.h5')
#load the tokenizer using pickle
with open('./models/tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

## Preprocessing

In [11]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shadow2030/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Text Cleanup

This function takes a raw text string and applies a standard NLP preprocessing pipeline consisting of the following steps:
- Text cleaning
- Tokenization
- Stopwords removal
- Stemming (Snowball stemmer)


In [12]:
def clean_text(text):
    
    # Text cleaning pipeline
    
    # 1. Text cleaning
    # 1.a Case normalization
    text = text.lower()
    
    # 1.b Trimming
    text = text.strip()
    
    # 1.c Filter out punctuation symbols
    text = re.sub("[^a-zA-Z\s]", "", text)
    
    # 1.d Filter out any internal extra whitespace
    text = re.sub(" +", " ", text)
    
    # 2. Tokenization (split text into tokens)
    tokens = text.split()
    
    # 3. Stopwords removal
    stop_words = set(stopwords.words("english"))
    terms = [token for token in tokens if token not in stop_words]
    
    # 4. Stemming (Snowball stemmer)
    stemmer = SnowballStemmer(language="english")
    terms_stemmed = [stemmer.stem(term) for term in terms]
    
    # Return the cleaned text
    return " ".join(terms_stemmed)


In [13]:
def prepareText(rawTweet):
    """
    This function takes a raw tweet and returns a list of cleaned tokens
    """
    #clean the raw tweet
    cleanTweet = clean_text(getTweetContent(rawTweet))
    #tokenize the clean tweet
    tokens = tokenizer.texts_to_sequences([cleanTweet])
    #pad the tokens
    paddedTokens = pad_sequences(tokens, maxlen=23, padding='post')
    return paddedTokens

## Function to change the tweet background

In [14]:
def predictLive():
    tweets = getTweets()
    for tweet in tweets:
        prediction = np.round(model.predict(prepareText(tweet))[0]) #1 disaster, 0 not disaster
        if prediction == 1:
            colorBackground(tweet, "red")
        else:
            colorBackground(tweet, "green")
    

## Input Handling

Now we need a way to let the user interact with the predictor. 
Whenever the user presses ENTER, the application will fetch the tweets nearby and will classify them accordingly

In [15]:
from pynput import keyboard

# Define the code to be executed when the key is pressed
def handle_key_press(key):
    if key == keyboard.Key.enter:
        print("Enter key pressed!")
        predictLive()

    elif key == keyboard.Key.esc:
        print("Escape key pressed!")
        # Stop listener
        return False
    elif key == keyboard.Key.shift_r:
        #take a screenshot
        #get time stamp 
        timestamp = time.strftime("%Y%m%d-%H%M%S")

        driver.save_screenshot(f"./screenshots/{timestamp}.png")
        print("Screenshot taken!")
    

# Create a listener for key press events
listener = keyboard.Listener(on_press=handle_key_press)

# Start the listener
listener.start()

# Keep the program running until a specific key is pressed (e.g., Escape)
listener.join()


Enter key pressed!
1/1 [==============================] - 0s 12ms/step
Enter key pressed!
1/1 [==============================] - 0s 12ms/step
Enter key pressed!
1/1 [==============================] - 0s 12ms/step
Enter key pressed!
1/1 [==============================] - 0s 12ms/step
Escape key pressed!
